# Internal Coordinates and B-matrix

## Defining bonds
Assuming a reasonable initial geometrical input, the identification of which atoms are bonded may, in most cases, be simply achieved using standard covalent radii.

In [34]:
import psi4
import numpy as np
mol = psi4.geometry("""
  pubchem:ammonium
""")
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom) 

# Determine the connectivity using standard covalent radii
import covRadii
from itertools import combinations, permutations
from math import sqrt
from optking import v3d
Natom = len(xyzGeom)
Z= [mol.Z(i) for i in range(Natom)]
C = np.zeros( (Natom, Natom), bool )
for i,j in combinations( range(Natom), 2):
  R = v3d.dist( xyzGeom[i], xyzGeom[j])
  Rcov = (covRadii.R[int(Z[i])] + covRadii.R[int(Z[j])])/0.52917720859
  if R < 1.3 * Rcov:
    C[i,j] = C[j,i] = True
print (C)


	Searching PubChem database for ammonium
	Found 1 result
Starting Geometry for Ammonium
[[  0.00000000e+00  -8.29781462e-35   3.92826956e-06]
 [  1.58424594e+00   4.85035429e-17   1.12021175e+00]
 [  4.85021276e-17  -1.58419971e+00  -1.12023904e+00]
 [ -1.58424594e+00  -4.85035429e-17   1.12021175e+00]
 [ -4.85021276e-17   1.58419971e+00  -1.12023904e+00]]
[[False  True  True  True  True]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]]


## Determining internal coordinates automatically
Once the connectivity is determined, the chemically relevant bonds, angles, and dihedral angles can be determined from the connected atoms.  Linear bends require special treatment that will not be shown here.  For this example, of ammonia there are no dihedral angles, but similarly for dihedral angles, non-linear sequences of atoms (A-B-C-D) are used to define dihedral coordinates which are included.

In [43]:
intcos = []
from optking import v3d
# add stretches by connectivity
from optking import stre
for i,j in combinations( range(Natom), 2 ):
  if C[i,j]:
    s = stre.STRE(i, j)
    if s not in intcos:
      intcos.append(s)

# add bends by connectivity
from optking import bend
for i,j in permutations( range(Natom), 2):
  if C[i,j]:
    for k in range(i+1,Natom):  # make i<k; the constructor checks too
      if C[j,k]:
        (check, val) = v3d.angle(xyzGeom[i], xyzGeom[j], xyzGeom[k])
        if not check: continue
        b = bend.BEND(i, j, k)
        if b not in intcos:
          intcos.append(b)
          
for i in intcos:
    print "%10s = %10.5f" % (i, i.q(xyzGeom))

    R(1,2) =    1.94028
    R(1,3) =    1.94027
    R(1,4) =    1.94028
    R(1,5) =    1.94027
  B(2,1,3) =    1.91064
  B(2,1,4) =    1.91065
  B(2,1,5) =    1.91064
  B(3,1,4) =    1.91064
  B(3,1,5) =    1.91060
  B(4,1,5) =    1.91064


In [33]:
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})

These internal coordinates have many advantages. They tend to produce a strongly diagonal second derivative matrix (Hessian), and they have force constants which are readily guessable.  For many years, the primary difficulty is that the simple procedure illustrated here generates too many coordinates. Non-linear molecules have 3N-5 internal degrees of freedom, but including the simple internals between all bonded atoms will generally produce more coordinates.

For ammonia, we have 10 coordinates, while the molecule has only 7 degrees of freedom.  Various alternative coordinate systems were explored, for example "delocalized coordinates", and "natural internal coordinates".  However, an elegant and robust solution turned out to be simply to work with a redudant set and make the necessary mathematical adjustments.  This approach was advanced by Peng and Ayala [reference].

The Wilson B matrix is the key element in typical transformations between internal and cartesian coordinates.  The B matrix is defined as the derivative of the internal coordinates with respect to cartesian coordinates, and has dimensions of internal coordinates by 3 * N. 

$$\textbf {B}_{ij} = \frac{\delta q_i}{\delta x_j}$$

Simple rearrangment allows for the conversion between small changes in cartesian geometry to internal coordinates
$$\textbf {B}_{ij} \delta x = \delta q$$

This does not allow for the direct conversion between the geometry in cartesians and in internals. However, this isn't particularly helpful, since our goal is usually to conver a change in internal coordinates to cartesians which is covered later in discussing the back transformation. To create the internal geometry, we need to calculate the distances and angles using simple geometry.

Lets start by defining a simple cartesian geometry for ammonia and then defining all of the internal coordinates for the molecule. Note that the number of internal coordinates is greater than the number of degrees of freedom.

In [ ]:
mol.set_multiplicity (1)
mol.set_molecular_charge(1)

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom)


from optking import stre, bend
r1 = stre.STRE(0,1)
r2 = stre.STRE(0,2)
r3 = stre.STRE(0,3)
r4 = stre.STRE(0,4)
b1 = bend.BEND(1,0,2)
b2 = bend.BEND(2,0,3)
b3 = bend.BEND(3,0,4)
b4 = bend.BEND(4,0,1)
b5 = bend.BEND(4,0,2)
b6 = bend.BEND(3,0,1)
intcos = (r1, r2, r3, r4, b1, b2, b3, b4, b5, b6)

print ("\nList of internal Coordinates in Bohr and Radians")
for i in range (len(intcos)):
    #q(xyzGeom) simply calls v3d.dist() for the two atoms in a stretch, and v3d.angle() for the three atoms in a bond angle
    print (" %d %s %.16f" % (i + 1, intcos[i], intcos[i].q(xyzGeom)))

#create an array for geometry in internals
qGeom = np.zeros(len(intcos), float)
for i in range (len(intcos)):
    qGeom[i] = intcos[i].q(xyzGeom)

print("\n Geometry in internals")    
printTools.printArray(qGeom)    

In [ ]:
from optking import intcosMisc
Bmatrix = intcosMisc.Bmat(intcos, xyz)
print('B matrix')
print(Bmatrix)

Our goal is to calculate the step in internal coordintes, this requires that either the forces or gradient are converted into internals.

In order to convert between the gradient or forces we need to introduce $\delta E$ luckily this easily simplifies to the form we need

$$\frac{\textbf {B}}{\delta q} = (\delta x)^{-1} \equiv \frac{\delta q}{\delta x} \frac{\delta E}{\delta q} = \frac{\delta E}{\delta x} \equiv \textbf{B}^T g_q = g_x$$ 

First lets get the gradient from Psi4, which is in cartesian

In [ ]:
psi4gradientMatrix = psi4.gradient('hf')
g = np.array( psi4gradientMatrix )
print (g)